<a href="https://colab.research.google.com/github/neuroidss/EEG-GAN-audio-video/blob/main/EEG_stream_kernel_comms_connectivity_Generate_Abstraction%26Anime_Portraits_with_StyleGAN2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
ch_names = ['FP1','AF3','F7','F3','FC1','FC5','T7','C3','CP1','CP5','P7','P3','Pz','PO3','O1','Oz','O2','PO4','P4','P8','CP6','CP2','C4','T8','FC6','FC2','F4','F8','AF4','FP2','Fz','Cz']
ch_locations=[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31]
##Bernard's 19ch:
#ch_names = ["FP2","FP1","O2","T6","T4","F8","F4","C4","P4","F3","C3","P3","O1","T5","T3","F7","FZ","PZ"]#,"other"]
#ch_locations=[4,24,0,1,2,3,5,6,7,25,26,27,28,29,30,31,16,12]#,8]
##Bernard's 2ch:
#ch_names = ["FP2","FP1"]#,"other"]
#ch_locations=[4,24]#,8]



In [2]:
bands = [[8.,12.]]
#bands = [[4.,7.],[8.,12.],[13.,29.]]
#bands = [[8.,12.],[8.,12.],[8.,12.]]
methods = ['coh']
#methods = ['ciplv']
#methods = ['coh', 'plv', 'ciplv', 'ppc', 'pli', 'wpli']

In [3]:
duration=5*1/8
overlap=duration-0.1
xsize=300
ysize=300

In [4]:
  stepSize = 1/pow(2,24)
vref = 2.50 #2.5V voltage ref +/- 250nV
gain = 8

vscale = (vref/gain)*stepSize #volts per step.
uVperStep = 1000000 * ((vref/gain)*stepSize) #uV per step.
scalar = 1/(1000000 / ((vref/gain)*stepSize)) #steps per uV.


In [5]:
%tensorflow_version 1.x
import tensorflow as tf

TensorFlow 1.x selected.


In [6]:
print('Tensorflow version: {}'.format(tf.__version__) )

Tensorflow version: 1.15.2


In [7]:
%cd /content
#!git clone https://github.com/NVlabs/stylegan2.git
!git clone https://github.com/NVlabs/stylegan2-ada.git


/content
Cloning into 'stylegan2-ada'...
remote: Enumerating objects: 71, done.
remote: Total 71 (delta 0), reused 0 (delta 0), pack-reused 71
Unpacking objects: 100% (71/71), done.


In [8]:
%cd /content/stylegan2-ada

/content/stylegan2-ada


In [9]:
!pip install googledrivedownloader

In [10]:
#import pretrained_networks
from google_drive_downloader import GoogleDriveDownloader as gdd

#url = 'https://drive.google.com/open?id=1WNQELgHnaqMTq3TlrnDaVkyrAH8Zrjez'
#anime2:
#url = 'https://drive.google.com/open?id=1YckI8gwqPbZBI8X4eaQAJCgWx-CqCTdi'
#anime:
#url = 'https://drive.google.com/open?id=1aUrChOhq5jDEddZK1v_Dp1vYNlHSBL9o'
#abstract:
url = 'https://drive.google.com/open?id=1ie1vWw1JNsfrZWRtMvhteqzVz4mt4KGa'
#'https://drive.google.com/open?id=1BHeqOZ58WZ-vACR2MJkh1ZVbJK2B-Kle'
model_id = url.replace('https://drive.google.com/open?id=', '')

network_pkl = '/content/models/model_%s.pkl' % model_id#(hashlib.md5(model_id.encode()).hexdigest())
gdd.download_file_from_google_drive(file_id=model_id,
                                    dest_path=network_pkl)
#_G, _D, Gs = pretrained_networks.load_networks(network_pkl)

import pickle

import dnnlib
import dnnlib.tflib as tflib
#
#tflib.init_tf()

#with dnnlib.util.open_url(network_pkl) as fp:
#        _G, _D, Gs = pickle.load(fp)


In [11]:
import dnnlib 
import dnnlib.tflib as tflib 
import PIL.Image 
from tqdm import tqdm

In [12]:
import numpy as np

In [13]:
!pip install tqdm

In [14]:
!pip install mne


     |████████████████████████████████| 6.9 MB 11.4 MB/s 


In [15]:
import mne
from mne import io
from mne.datasets import sample
from mne.minimum_norm import read_inverse_operator, compute_source_psd

from mne.connectivity import spectral_connectivity, seed_target_indices

import pandas as pd
import numpy as np


In [16]:
    sfreq = 512 
    ch_types=['eeg']*len(ch_names)
    info = mne.create_info(ch_names = ch_names, sfreq = sfreq, ch_types=ch_types)

In [17]:
!pip install imageio==2.4.1
!pip install imageio-ffmpeg==0.4.3 pyspng==0.1.0

     |████████████████████████████████| 26.9 MB 84 kB/s 
     |████████████████████████████████| 195 kB 48.1 MB/s 


In [18]:
!mkdir '/content/out'

In [19]:
label_names = ch_names

no_names = [''] * len(label_names)

In [20]:
tflib.init_tf()
with dnnlib.util.open_url(network_pkl) as fp:
     _G, _D, Gs = pickle.load(fp)


In [24]:
from IPython.display import Javascript
from IPython.display import display, Javascript
from google.colab.output import eval_js
from base64 import b64decode
from IPython.display import Javascript
import json
data_read=False


from IPython.display import Image
import matplotlib.figure
import base64
from io import BytesIO

data_read=None
data_buffer=None
data_analyse=None

import imageio
fps=3
out = imageio.get_writer('/content/out/output.mp4', mode='I', fps=fps, codec='libx264', bitrate='16M')

js_image = ""

Gs_kwargs = dnnlib.EasyDict()
Gs_kwargs.output_transform = dict(func=tflib.convert_images_to_uint8, nchw_to_nhwc=True)
Gs_kwargs.randomize_noise = False

samples=0

from time import perf_counter
time100=perf_counter()
time111=perf_counter()

from PIL import ImageFont, ImageDraw
last_time=0
def target_func1(comm, msg):
  time000=perf_counter()
  #import dnnlib 
  #import dnnlib.tflib as tflib 
  #import PIL.Image 
  #from tqdm import tqdm
  global _G, _D, Gs
  #tflib.init_tf()
  #with dnnlib.util.open_url(network_pkl) as fp:
  #   _G, _D, Gs = pickle.load(fp)

  # Only send the response if it's the data we are expecting.
  if msg['content']['data'] == 'the data':
    comm.send({
          'response': 'got comm open!',
        }, None, msg['buffers']);
    print(msg['buffers'])
  #else:
    #print(msg['content']['data'])
    #data_read=True
  global last_time
  global time100
  global samples
  array_to_receive_as_json = msg['content']['data']
  #global data_read
  data_read = json.loads(array_to_receive_as_json)
  #print('got data')
  #print(data_read)
  new_data_len=len(data_read[0])
  #print(new_data_len)
  samples=samples+new_data_len#len(data_analyse[0])
  time110=perf_counter()
  out_text=f'{(samples):7.0f}'+' sam, '
  out_text=out_text+f'{(new_data_len):4.0f}'+" sam/pac, "
  out_text=out_text+f'{(time110-time100):7.2f}'+" sec, "
  out_text=out_text+f'{(samples/(time110-time100)):7.2f}'+' sam/sec'
  out_text=out_text+f'{(time110-time100)-last_time:7.2f}'+" sec, "
  out_text=out_text+f'{(new_data_len/((time110-time100)-last_time)):7.2f}'+' sam/sec'
  last_time=(time110-time100)
  #comm.send({
  #        'response': out_text,
  #      }, None, msg['buffers']);
  #print(out_text)
  global label_names,ch_locations,info,bands,duration,overlap,methods,uVperStep,sfreq
  
  if not (data_read is None):
      #print('read_data-data_read')
      #print(len(data_read[0]))
      global data_buffer
      if data_buffer is None:
        data_buffer=data_read
      else:
        data_buffer=np.concatenate((data_buffer,data_read),axis=1)
      data_read = None

      data_buffer_len=len(data_buffer[0])
      #print(data_buffer_len)
      data_slice_len=int(duration*512)
      #print(data_slice_len)
      if data_buffer_len>data_slice_len:
        data_buffer=data_buffer[0:len(data_buffer),data_buffer_len-data_slice_len:data_buffer_len]
      data_analyse=data_buffer
      #data_buffer=data_analyse
      #print('read_data-data_analyse')
      #print(len(data_analyse[0]))

  
  #if False: 
  if True: 
    #if (time111-time000)>=(1/fps):
      time111=perf_counter()
      data_uv = [0]*len(label_names)
      for j in range(len(label_names)):
        data_uv[j]=[0]*len(data_analyse[ch_locations[j]])
        for i in range(len(data_analyse[ch_locations[j]])):
          data_uv[j][i] = data_analyse[ch_locations[j]][i] * uVperStep * 2
  
      time002=perf_counter()
      raw = mne.io.RawArray(data_uv, info, verbose=50)
      time003=perf_counter()
      #print (f'003: {(time003-time000):.1f}s')
      datas=[]
      for band in range(len(bands)):
        datas.append(raw)
      time004=perf_counter()
      epochs = []
      #print (f'004: {(time004-time000):.1f}s')
      for band in range(len(bands)):
        epochs.append(mne.make_fixed_length_epochs(datas[band], duration=duration, preload=True, overlap=overlap, verbose=50))
      time005=perf_counter()
      #print (f'005: {(time005-time000):.1f}s')
      n_generate=int((float(len(data_uv[0]))/float(sfreq))/duration)
      n_generate=1
      part_len = 10
      dim = 512

      n_parts = n_generate//part_len
      if n_generate%part_len>0:
        n_parts=n_parts+1
    
      vol=1
      #vol=0.1
    
      psd_array=np.random.rand(part_len, dim) 
      time006=perf_counter()
      #print (f'006: {(time006-time000):.1f}s')
    
      for j in range(n_parts): # display separate audio for each break
        for i in range(part_len): # display separate audio for each break
            ji = j * part_len + i
            
            if (i==0) and (n_generate-ji<part_len):
                psd_array=np.random.rand((n_generate-ji), dim) 
        
            sfreq = raw.info['sfreq']  # the sampling frequency
            
            psds=np.zeros(dim)
            
            for method in range(len(methods)):
             for band in range(len(bands)):
              fmin=bands[band][0]
              fmax=bands[band][1]
              if band == 0:
               con, freqs, times, n_epochs, n_tapers = spectral_connectivity(
                epochs[band][ji:ji+1], method=methods[method], mode='multitaper', sfreq=sfreq, fmin=fmin,
                fmax=fmax, faverage=True, mt_adaptive=True, n_jobs=10, verbose=50)
              psds_shift1=int(round(method*len(bands)+band)*(len(ch_names)*(len(ch_names)-1)/2))
              ji1=0
              for j1 in range(0,len(ch_names)): # display separate audio for each break
                for i1 in range(0,j1): # display separate audio for each break
                  psds[ji1+psds_shift1]=(con[j1][i1][0]-0.5)*1
                  ji1 = ji1+1
            psd_array[i]=psds
            if (i==part_len-1) or (ji==n_generate-1) :
                _z = psd_array * vol
                images = Gs.run(_z, None, **Gs_kwargs) # [minibatch, height, width, channel]
                for image in images:
                  image_pil=PIL.Image.fromarray(image, 'RGB')
                  image_asarray=np.asarray(image_pil)
                  out.append_data(image_asarray)
                  img=image_pil.resize((xsize,ysize),PIL.Image.ANTIALIAS)
                  buffer = BytesIO()
                  img.save(buffer,format="JPEG")                  #Enregistre l'image dans le buffer
                  #img.save(buffer,format="PNG")                  #Enregistre l'image dans le buffer
                  buffer.seek(0)
                  myimage = buffer.getvalue()   
                  encoded= "data:image/jpeg;base64,"+base64.b64encode(myimage).decode()
                  #encoded= "data:image/png;base64,"+base64.b64encode(myimage).decode()
                  js='''displayPhoto1("{0}",{1},{2})'''.format(encoded,xsize,ysize)
                if (ji==n_generate-1) :
                    time110=perf_counter()
                    #print (f'110: {(time110-time000):.1f}s')
                    #js_image=js
                    comm.send({
                      'response': encoded,
                      }, None, msg['buffers']);
                    break

    #if not(js_image==""):
    #  js=js_image
    #  js_image=""
    #  eval_js(js)
#      if True:
      if False:
                    time111=perf_counter()
                    print (f'000: {(time111-time000):.1f}s, '+
                      f'001: {(time111-time001):.1f}s, '+
                      f'110: {(time111-time110):.1f}s, '+
                      f'001-000: {(time001-time000):.1f}s, {(time001-time000)*100/(time111-time000):.0f}%, '+
                      f'110-001: {(time110-time001):.1f}s, {(time110-time001)*100/(time111-time000):.0f}%. '+
                      f'111-110: {(time111-time110):.1f}s, {(time111-time110)*100/(time111-time000):.0f}%')





data_read=False
get_ipython().kernel.comm_manager.register_target('comm_target1', target_func1)
#for i in range(100):
#  get_ipython().kernel.comm_manager.register_target(str(i), target_func1)

Javascript('''


//Joshua Brewster, GPL (copyleft)

//import 'regenerator-runtime/runtime' //For async functions on node\\

 class eeg32 { //Contains structs and necessary functions/API calls to analyze serial data for the FreeEEG32

    constructor(
		onDecodedCallback = this.onDecodedCallback,
		onConnectedCallback = this.onConnectedCallback,
		onDisconnectedCallback = this.onDisconnectedCallback,
		CustomDecoder = this.decode,
		baudrate = 921600//115200
		) {

		this.onDecodedCallback = onDecodedCallback;
		this.onConnectedCallback = onConnectedCallback;
		this.onDisconnectedCallback = onDisconnectedCallback;
		this.decode = CustomDecoder;
		//Free EEG 32 data structure:
        
        //    [stop byte, start byte, counter byte, 32x3 channel data bytes (24 bit), 3x2 accelerometer data bytes, stop byte, start byte...] Gyroscope not enabled yet but would be printed after the accelerometer..
        //    Total = 105 bytes/line
        
		this.connected = false;
		this.subscribed = false;
        this.buffer = [];
        this.startByte = 160; // Start byte value
		this.stopByte = 192; // Stop byte value
		this.searchString = new Uint8Array([this.stopByte,this.startByte]); //Byte search string
		this.readRate = 16.666667; //Throttle EEG read speed. (1.953ms/sample min @103 bytes/line)
		this.readBufferSize = 2000; //Serial read buffer size, increase for slower read speeds (~1030bytes every 20ms) to keep up with the stream (or it will crash)

		this.sps = 512; // Sample rate
		this.nChannels = 32;
		this.nPeripheralChannels = 6; // accelerometer and gyroscope (2 bytes * 3 coordinates each)
		this.updateMs = 1000/this.sps; //even spacing
		this.stepSize = 1/Math.pow(2,24);
		this.vref = 2.50; //2.5V voltage ref +/- 250nV
		this.gain = 8;

		this.vscale = (this.vref/this.gain)*this.stepSize; //volts per step.
		this.uVperStep = 1000000 * ((this.vref/this.gain)*this.stepSize); //uV per step.
		this.scalar = 1/(1000000 / ((this.vref/this.gain)*this.stepSize)); //steps per uV.

		this.maxBufferedSamples = this.sps*60*2; //max samples in buffer this.sps*60*nMinutes = max minutes of data
		
		this.data = { //Data object to keep our head from exploding. Get current data with e.g. this.data.A0[this.data.count-1]
			count: 0,
			startms: 0,
			ms: [],
			'A0': [],'A1': [],'A2': [],'A3': [],'A4': [],'A5': [],'A6': [],'A7': [], //ADC 0
			'A8': [],'A9': [],'A10': [],'A11': [],'A12': [],'A13': [],'A14': [],'A15': [], //ADC 1
			'A16': [],'A17': [],'A18': [],'A19': [],'A20': [],'A21': [],'A22': [],'A23': [], //ADC 2
			'A24': [],'A25': [],'A26': [],'A27': [],'A28': [],'A29': [],'A30': [],'A31': [], //ADC 3
			'Ax': [], 'Ay': [], 'Az': [], 'Gx': [], 'Gy': [], 'Gz': []  //Peripheral data (accelerometer, gyroscope)
		};

    this.bufferednewLines = 0;
    this.data_slice=[];
    this.data_slice_size=512*(5*1/8+0.1);
    this.ready_to_send_data = false;
    this.data_send_count=0;
    this.fps=2;
    this.samples_count=0;
    //this.channel=None;

		this.resetDataBuffers();

		//navigator.serial utils
		if(!navigator.serial){
			console.error("`navigator.serial not found! Enable #enable-experimental-web-platform-features in chrome://flags (search 'experimental')")
		}
		this.port = null;
		this.reader = null;
		this.baudrate = baudrate;

	}
	
	resetDataBuffers(){
		this.data.count = 0;
		this.data.startms = 0;
		for(const prop in this.data) {
			if(typeof this.data[prop] === "object"){
				this.data[prop] = new Array(this.maxBufferedSamples).fill(0);
			}
		}
	}

	setScalar(gain=24,stepSize=1/(Math.pow(2,23)-1),vref=4.50) {
        this.stepSize = stepSize;
		this.vref = vref; //2.5V voltage ref +/- 250nV
		this.gain = gain;

		this.vscale = (this.vref/this.gain)*this.stepSize; //volts per step.
		this.uVperStep = 1000000 * ((this.vref/this.gain)*this.stepSize); //uV per step.
		this.scalar = 1/(1000000 / ((this.vref/this.gain)*this.stepSize)); //steps per uV.
    }

	getLatestData(channel="A0",count=1) { //Return slice of specified size of the latest data from the specified channel
		let ct = count;
		if(ct <= 1) {
			return [this.data[channel][this.data.count-1]];
		}
		else {
			if(ct > this.data.count) {
				ct = this.data.count;
			}
			return this.data[channel].slice(this.data.count-ct,this.data.count);
		}
	}

    bytesToInt16(x0,x1){
		return x0 * 256 + x1;
    }

    int16ToBytes(y){ //Turns a 24 bit int into a 3 byte sequence
        return [y & 0xFF , (y >> 8) & 0xFF];
    }

    bytesToInt24(x0,x1,x2){ //Turns a 3 byte sequence into a 24 bit int
        return x0 * 65536 + x1 * 256 + x2;
    }

    int24ToBytes(y){ //Turns a 24 bit int into a 3 byte sequence
        return [y & 0xFF , (y >> 8) & 0xFF , (y >> 16) & 0xFF];
    }

    decode(buffer = this.buffer) { //returns true if successful, returns false if not

		var needle = this.searchString
		var haystack = buffer;
		var search = this.boyerMoore(needle);
		var skip = search.byteLength;
		var indices = [];
		let newLines = 0;
    
		for (var i = search(haystack); i !== -1; i = search(haystack, i + skip)) {
			indices.push(i);
		}
		//console.log(indices);
		if(indices.length >= 2){
			for(let k = 1; k < indices.length; k++) {
				if(indices[k] - indices[k-1] !== 105) {
					
				} //This is not a valid sequence going by size, drop sequence and return
				else {
					var line = buffer.slice(indices[k-1],indices[k]+1); //Splice out this line to be decoded
					
					// line[0] = stop byte, line[1] = start byte, line[2] = counter, line[3:99] = ADC data 32x3 bytes, line[100-104] = Accelerometer data 3x2 bytes

					//line found, decode.
					if(this.data.count < this.maxBufferedSamples){
						this.data.count++;
					}

					if(this.data.count-1 === 0) {this.data.ms[this.data.count-1]= Date.now(); this.data.startms = this.data.ms[0];}
					else {
						this.data.ms[this.data.count-1]=this.data.ms[this.data.count-2]+this.updateMs;
						
						if(this.data.count >= this.maxBufferedSamples) {
							this.data.ms.splice(0,5120);
							this.data.ms.push(new Array(5120).fill(0));
						}
					}//Assume no dropped samples
				  var sample_count = line[2];
				  var sample_count_diff = sample_count-this.samples_count;
          if(sample_count_diff<0){
            sample_count_diff+=256;
          }
          if(sample_count_diff!=1)
          {
            console.error("dropped samples:"+sample_count_diff.toString());
          }
          this.samples_count=sample_count;

					for(var i = 3; i < 99; i+=3) {
						var channel = "A"+(i-3)/3;
						this.data[channel][this.data.count-1]=this.bytesToInt24(line[i],line[i+1],line[i+2]);
						if(this.data.count >= this.maxBufferedSamples) { 
							this.data[channel].splice(0,5120);
							this.data[channel].push(new Array(5120).fill(0));//shave off the last 10 seconds of data if buffer full (don't use shift())
						}
							//console.log(this.data[channel][this.data.count-1],indices[k], channel)
					}

					this.data["Ax"][this.data.count-1]=this.bytesToInt16(line[99],line[100]);
					this.data["Ay"][this.data.count-1]=this.bytesToInt16(line[101],line[102]);
					this.data["Az"][this.data.count-1]=this.bytesToInt16(line[103],line[104]);

					
					if(this.data.count >= this.maxBufferedSamples) { 
						this.data["Ax"].splice(0,5120);
						this.data["Ay"].splice(0,5120);
						this.data["Az"].splice(0,5120);
						this.data["Ax"].push(new Array(5120).fill(0))
						this.data["Ay"].push(new Array(5120).fill(0))
						this.data["Az"].push(new Array(5120).fill(0))
						this.data.count -= 5120;
					}
					//console.log(this.data)
					newLines++;
					//console.log(indices[k-1],indices[k])
					//console.log(buffer[indices[k-1],buffer[indices[k]]])
					//indices.shift();
				}
				
			}
			if(newLines > 0) buffer.splice(0,indices[indices.length-1]);
   
			return newLines;
			//Continue
		}
		//else {this.buffer = []; return false;}
	}
	//Callbacks
	onDecodedCallback(newLinesInt){
		//console.log("new samples:", newLinesInt);
    this.bufferednewLines=this.bufferednewLines+newLinesInt;
    if(this.bufferednewLines>this.data_slice_size)
      {
        this.bufferednewLines=this.data_slice_size;
      }
    if(this.bufferednewLines>512/this.fps)
    {

        this.takeandsendSlice(this.data.count-1-this.bufferednewLines,this.data.count-1);
    }
		//console.log("this.bufferednewLines:", this.bufferednewLines);
    //this.takeSlice(this.data.count-1-this.bufferednewLines,this.data.count-1);
    //this.takeandsendSlice(this.data.count-1-this.bufferednewLines,this.data.count-1);
    //this.takeandsendSliceBroadcast(this.data.count-1-this.bufferednewLines,this.data.count-1);
 	}

	onConnectedCallback() {
		console.log("port connected!");
	}

	onDisconnectedCallback() {
		console.log("port disconnected!");
	}

	onReceive(value){
		this.buffer.push(...value);

		let newLines = this.decode(this.buffer);
		//console.log(this.data)
		//console.log("decoding... ", this.buffer.length)
		if(newLines !== false && newLines !== 0 && !isNaN(newLines) ) this.onDecodedCallback(newLines);
	}

	async onPortSelected(port,baud=this.baudrate) {
		try{
			try {
				await port.open({ baudRate: baud, bufferSize: this.readBufferSize });
				this.onConnectedCallback();
				this.connected = true;
				this.subscribed = true;
				this.subscribe(port);//this.subscribeSafe(port);
		
			} //API inconsistency in syntax between linux and windows
			catch {
				await port.open({ baudrate: baud, buffersize: this.readBufferSize });
				this.onConnectedCallback();
				this.connected = true;
				this.subscribed = true;
				this.subscribe(port);//this.subscribeSafe(port);
			}
		}
		catch(err){
			console.log(err);
			this.connected = false;
		}
	}

	async subscribe(port){
		if (this.port.readable && this.subscribed === true) {
			this.reader = port.readable.getReader();
			const streamData = async () => {
				try {
					const { value, done } = await this.reader.read();
					if (done || this.subscribed === false) {
						// Allow the serial port to be closed later.
						await this.reader.releaseLock();
						
					}
					if (value) {
						//console.log(value.length);
						try{
							this.onReceive(value);
						}
						catch (err) {console.log(err)}
						//console.log("new Read");
						//console.log(this.decoder.decode(value));
					}
					if(this.subscribed === true) {
						setTimeout(()=>{streamData();}, this.readRate);//Throttled read 1/512sps = 1.953ms/sample @ 103 bytes / line or 1030bytes every 20ms
					}
				} catch (error) {
					console.log(error);// TODO: Handle non-fatal read error.
                    if(error.message.includes('framing') || error.message.includes('overflow') || error.message.includes('Overflow') || error.message.includes('break')) {
                        this.subscribed = false;
                        setTimeout(async ()=>{
							try{
                            if (this.reader) {
                                await this.reader.releaseLock();
                                this.reader = null;
                            }
							} catch (er){ console.error(er);}
                            this.subscribed = true; 
                            this.subscribe(port);
                            //if that fails then close port and reopen it
                        },30); //try to resubscribe 
                    } else if (error.message.includes('parity') || error.message.includes('Parity') || error.message.includes('overrun') ) {
                        if(this.port){
                            this.subscribed = false;
                            setTimeout(async () => {
								try{
                                if (this.reader) {
                                    await this.reader.releaseLock();
                                    this.reader = null;
                                }
                                await port.close();
								} catch (er){ console.error(er);}
                                //this.port = null;
                                this.connected = false;
                                setTimeout(()=>{this.onPortSelected(this.port)},100); //close the port and reopen
                            }, 50);
                        }
                    }
                     else {
                        this.closePort();	
                    }	
				}
			}
			streamData();
		}
	}

	//Unfinished
	async subscribeSafe(port) { //Using promises instead of async/await to cure hangs when the serial update does not meet tick requirements
		var readable = new Promise((resolve,reject) => {
			while(this.port.readable && this.subscribed === true){
				this.reader = port.readable.getReader();
				var looper = true;
				var prom1 = new Promise((resolve,reject) => {
					return this.reader.read();
				});

				var prom2 = new Promise((resolve,reject) => {
					setTimeout(resolve,100,"readfail");
				});
				while(looper === true ) {
					//console.log("reading...");
					Promise.race([prom1,prom2]).then((result) => {
						console.log("newpromise")
						if(result === "readfail"){
							console.log(result);
						}
						else{
							const {value, done} = result;
							if(done === true || this.subscribed === true) { var donezo = new Promise((resolve,reject) => {
								resolve(this.reader.releaseLock())}).then(() => {
									looper = false;
									return;
								});
							}
							else{
								this.onReceive(value);
							}
						}
					});
				}
			}
			resolve("not readable");
		});
	}

	async closePort(port=this.port) {
		//if(this.reader) {this.reader.releaseLock();}
		if(this.port){
			this.subscribed = false;
			setTimeout(async () => {
				if (this.reader) {
					await this.reader.releaseLock();
					this.reader = null;
				}
				await port.close();
				this.port = null;
				this.connected = false;
				this.onDisconnectedCallback();
			}, 100);
		}
	}

	async setupSerialAsync(baudrate=this.baudrate) { //You can specify baudrate just in case

		const filters = [
			{ usbVendorId: 0x10c4, usbProductId: 0x0043 } //CP2102 filter (e.g. for UART via ESP32)
		];

		this.port = await navigator.serial.requestPort();
		navigator.serial.addEventListener("disconnect",(e) => {
			this.closePort(this.port);
		});
		this.onPortSelected(this.port,baudrate);

		//navigator.serial.addEventListener("onReceive", (e) => {console.log(e)});//this.onReceive(e));

	}


	//Boyer Moore fast byte search method copied from https://codereview.stackexchange.com/questions/20136/uint8array-indexof-method-that-allows-to-search-for-byte-sequences
	asUint8Array(input) {
		if (input instanceof Uint8Array) {
			return input;
		} else if (typeof(input) === 'string') {
			// This naive transform only supports ASCII patterns. UTF-8 support
			// not necessary for the intended use case here.
			var arr = new Uint8Array(input.length);
			for (var i = 0; i < input.length; i++) {
			var c = input.charCodeAt(i);
			if (c > 127) {
				throw new TypeError("Only ASCII patterns are supported");
			}
			arr[i] = c;
			}
			return arr;
		} else {
			// Assume that it's already something that can be coerced.
			return new Uint8Array(input);
		}
	}

	boyerMoore(patternBuffer) {
		// Implementation of Boyer-Moore substring search ported from page 772 of
		// Algorithms Fourth Edition (Sedgewick, Wayne)
		// http://algs4.cs.princeton.edu/53substring/BoyerMoore.java.html
		
//		USAGE:
			// needle should be ASCII string, ArrayBuffer, or Uint8Array
			// haystack should be an ArrayBuffer or Uint8Array
//			var search = boyerMoore(needle);
//			var skip = search.byteLength;
//			var indices = [];
//			for (var i = search(haystack); i !== -1; i = search(haystack, i + skip)) {
//				indices.push(i);
//			}
		
		var pattern = this.asUint8Array(patternBuffer);
		var M = pattern.length;
		if (M === 0) {
			throw new TypeError("patternBuffer must be at least 1 byte long");
		}
		// radix
		var R = 256;
		var rightmost_positions = new Int32Array(R);
		// position of the rightmost occurrence of the byte c in the pattern
		for (var c = 0; c < R; c++) {
			// -1 for bytes not in pattern
			rightmost_positions[c] = -1;
		}
		for (var j = 0; j < M; j++) {
			// rightmost position for bytes in pattern
			rightmost_positions[pattern[j]] = j;
		}
		var boyerMooreSearch = (txtBuffer, start, end) => {
			// Return offset of first match, -1 if no match.
			var txt = this.asUint8Array(txtBuffer);
			if (start === undefined) start = 0;
			if (end === undefined) end = txt.length;
			var pat = pattern;
			var right = rightmost_positions;
			var lastIndex = end - pat.length;
			var lastPatIndex = pat.length - 1;
			var skip;
			for (var i = start; i <= lastIndex; i += skip) {
				skip = 0;
				for (var j = lastPatIndex; j >= 0; j--) {
				var c = txt[i + j];
				if (pat[j] !== c) {
					skip = Math.max(1, j - right[c]);
					break;
				}
				}
				if (skip === 0) {
				return i;
				}
			}
			return -1;
		};
		boyerMooreSearch.byteLength = pattern.byteLength;
		return boyerMooreSearch;
	}
	//---------------------end copy/pasted solution------------------------

    async  takeSlice(data_slice_from,data_slice_to) {
      this.data_slice = [
        //device.data.ms.slice(data_slice_from,data_slice_to),
        this.data["A"+0].slice(data_slice_from,data_slice_to),
        this.data["A"+1].slice(data_slice_from,data_slice_to),
        this.data["A"+2].slice(data_slice_from,data_slice_to),
        this.data["A"+3].slice(data_slice_from,data_slice_to),
        this.data["A"+4].slice(data_slice_from,data_slice_to),
        this.data["A"+5].slice(data_slice_from,data_slice_to),
        this.data["A"+6].slice(data_slice_from,data_slice_to),
        this.data["A"+7].slice(data_slice_from,data_slice_to),
        this.data["A"+8].slice(data_slice_from,data_slice_to),
        this.data["A"+9].slice(data_slice_from,data_slice_to),
        this.data["A"+10].slice(data_slice_from,data_slice_to),
        this.data["A"+11].slice(data_slice_from,data_slice_to),
        this.data["A"+12].slice(data_slice_from,data_slice_to),
        this.data["A"+13].slice(data_slice_from,data_slice_to),
        this.data["A"+14].slice(data_slice_from,data_slice_to),
        this.data["A"+15].slice(data_slice_from,data_slice_to),
        this.data["A"+16].slice(data_slice_from,data_slice_to),
        this.data["A"+17].slice(data_slice_from,data_slice_to),
        this.data["A"+18].slice(data_slice_from,data_slice_to),
        this.data["A"+19].slice(data_slice_from,data_slice_to),
        this.data["A"+20].slice(data_slice_from,data_slice_to),
        this.data["A"+21].slice(data_slice_from,data_slice_to),
        this.data["A"+22].slice(data_slice_from,data_slice_to),
        this.data["A"+23].slice(data_slice_from,data_slice_to),
        this.data["A"+24].slice(data_slice_from,data_slice_to),
        this.data["A"+25].slice(data_slice_from,data_slice_to),
        this.data["A"+26].slice(data_slice_from,data_slice_to),
        this.data["A"+27].slice(data_slice_from,data_slice_to),
        this.data["A"+28].slice(data_slice_from,data_slice_to),
        this.data["A"+29].slice(data_slice_from,data_slice_to),
        this.data["A"+30].slice(data_slice_from,data_slice_to),
        this.data["A"+31].slice(data_slice_from,data_slice_to)
        ];
        //this.lastnewLines=this.bufferednewLines;
    }
    async  takeandsendSlice(data_slice_from,data_slice_to) {
        //this.lastnewLines=this.bufferednewLines;
     //if(this.ready_to_send_data&&this.bufferednewLines){//&&this.data_slice[0].length)
     if(this.ready_to_send_data&&this.bufferednewLines)//&&this.data_slice[0].length)
     {
      //this.ready_to_send_data = false;
      this.data_slice = [
        //device.data.ms.slice(data_slice_from,data_slice_to),
        this.data["A"+0].slice(data_slice_from,data_slice_to),
        this.data["A"+1].slice(data_slice_from,data_slice_to),
        this.data["A"+2].slice(data_slice_from,data_slice_to),
        this.data["A"+3].slice(data_slice_from,data_slice_to),
        this.data["A"+4].slice(data_slice_from,data_slice_to),
        this.data["A"+5].slice(data_slice_from,data_slice_to),
        this.data["A"+6].slice(data_slice_from,data_slice_to),
        this.data["A"+7].slice(data_slice_from,data_slice_to),
        this.data["A"+8].slice(data_slice_from,data_slice_to),
        this.data["A"+9].slice(data_slice_from,data_slice_to),
        this.data["A"+10].slice(data_slice_from,data_slice_to),
        this.data["A"+11].slice(data_slice_from,data_slice_to),
        this.data["A"+12].slice(data_slice_from,data_slice_to),
        this.data["A"+13].slice(data_slice_from,data_slice_to),
        this.data["A"+14].slice(data_slice_from,data_slice_to),
        this.data["A"+15].slice(data_slice_from,data_slice_to),
        this.data["A"+16].slice(data_slice_from,data_slice_to),
        this.data["A"+17].slice(data_slice_from,data_slice_to),
        this.data["A"+18].slice(data_slice_from,data_slice_to),
        this.data["A"+19].slice(data_slice_from,data_slice_to),
        this.data["A"+20].slice(data_slice_from,data_slice_to),
        this.data["A"+21].slice(data_slice_from,data_slice_to),
        this.data["A"+22].slice(data_slice_from,data_slice_to),
        this.data["A"+23].slice(data_slice_from,data_slice_to),
        this.data["A"+24].slice(data_slice_from,data_slice_to),
        this.data["A"+25].slice(data_slice_from,data_slice_to),
        this.data["A"+26].slice(data_slice_from,data_slice_to),
        this.data["A"+27].slice(data_slice_from,data_slice_to),
        this.data["A"+28].slice(data_slice_from,data_slice_to),
        this.data["A"+29].slice(data_slice_from,data_slice_to),
        this.data["A"+30].slice(data_slice_from,data_slice_to),
        this.data["A"+31].slice(data_slice_from,data_slice_to)
        ];
  this.bufferednewLines=0;
  //const buffer = new Uint8Array(10);
  //for (let i = 0; i < buffer.byteLength; ++i) {
  //  buffer[i] = i
  //}
  var array_to_send_as_json = JSON.stringify(this.data_slice);
  //document.body.appendChild(document.createTextNode('sending ready'));
  this.data_send_count++;
  //if(this.channel==None)
  //{
  //  this.channel = await google.colab.kernel.comms.open('comm_target1', array_to_send_as_json, []);
    //this.channel = await google.colab.kernel.comms.open(this.data_send_count.toString(), array_to_send_as_json, []);
  //} else 
  //{
    //this.channel.send(this.data_send_count.toString())
  //}
  const channel = await google.colab.kernel.comms.open('comm_target1', array_to_send_as_json, []);
  //const channel = await google.colab.kernel.comms.open('comm_target1', 'the data', [buffer.buffer]);
  let success = false;
  for await (const message of channel.messages) {
    if (message.data.response == 'got comm open!') {
      const responseBuffer = new Uint8Array(message.buffers[0]);
      for (let i = 0; i < buffer.length; ++i) {
        if (responseBuffer[i] != buffer[i]) {
          console.error('comm buffer different at ' + i);
          document.body.appendChild(document.createTextNode('comm buffer different at2 ' + i));
          return;
        }
      }
      // Close the channel once the expected message is received. This should
      // cause the messages iterator to complete and for the for-await loop to
      // end.
      //console.error('comm buffer same ' + responseBuffer);
      //document.body.appendChild(document.createTextNode('comm buffer same2 ' + responseBuffer));
      channel.close();
    }
    await displayPhoto1(message.data.response);
  }
      //this.ready_to_send_data = true;
  document.body.appendChild(document.createTextNode('done2.'));
     } 

    }
    async  takeandsendSliceBroadcast(data_slice_from,data_slice_to) {
      this.data_slice = [
        //device.data.ms.slice(data_slice_from,data_slice_to),
        this.data["A"+0].slice(data_slice_from,data_slice_to),
        this.data["A"+1].slice(data_slice_from,data_slice_to),
        this.data["A"+2].slice(data_slice_from,data_slice_to),
        this.data["A"+3].slice(data_slice_from,data_slice_to),
        this.data["A"+4].slice(data_slice_from,data_slice_to),
        this.data["A"+5].slice(data_slice_from,data_slice_to),
        this.data["A"+6].slice(data_slice_from,data_slice_to),
        this.data["A"+7].slice(data_slice_from,data_slice_to),
        this.data["A"+8].slice(data_slice_from,data_slice_to),
        this.data["A"+9].slice(data_slice_from,data_slice_to),
        this.data["A"+10].slice(data_slice_from,data_slice_to),
        this.data["A"+11].slice(data_slice_from,data_slice_to),
        this.data["A"+12].slice(data_slice_from,data_slice_to),
        this.data["A"+13].slice(data_slice_from,data_slice_to),
        this.data["A"+14].slice(data_slice_from,data_slice_to),
        this.data["A"+15].slice(data_slice_from,data_slice_to),
        this.data["A"+16].slice(data_slice_from,data_slice_to),
        this.data["A"+17].slice(data_slice_from,data_slice_to),
        this.data["A"+18].slice(data_slice_from,data_slice_to),
        this.data["A"+19].slice(data_slice_from,data_slice_to),
        this.data["A"+20].slice(data_slice_from,data_slice_to),
        this.data["A"+21].slice(data_slice_from,data_slice_to),
        this.data["A"+22].slice(data_slice_from,data_slice_to),
        this.data["A"+23].slice(data_slice_from,data_slice_to),
        this.data["A"+24].slice(data_slice_from,data_slice_to),
        this.data["A"+25].slice(data_slice_from,data_slice_to),
        this.data["A"+26].slice(data_slice_from,data_slice_to),
        this.data["A"+27].slice(data_slice_from,data_slice_to),
        this.data["A"+28].slice(data_slice_from,data_slice_to),
        this.data["A"+29].slice(data_slice_from,data_slice_to),
        this.data["A"+30].slice(data_slice_from,data_slice_to),
        this.data["A"+31].slice(data_slice_from,data_slice_to)
        ];
        //this.lastnewLines=this.bufferednewLines;
(async () => {
 if(this.ready_to_send_data&&this.bufferednewLines&&this.data_slice[0].length){
      this.ready_to_send_data = false;
  var array_to_send_as_json = JSON.stringify(this.data_slice);
  document.body.appendChild(document.createTextNode('sending ready'));
      const senderChannel = new BroadcastChannel("channel");
      senderChannel.postMessage(array_to_send_as_json);
  document.body.appendChild(document.createTextNode('done2.'));
  this.bufferednewLines=0;
 } 
})()

    }

}

device = new eeg32();

    connect = async () => {
        await this.device.setupSerialAsync();
    }

    disconnect = () => {
        if (this.ui) this.ui.deleteNode()
        this.device.closePort();
    }


        const canvas = document.createElement('canvas');

          var ctx = canvas.getContext("2d");

      var image = new Image();
      image.onload = function() {
        ctx.drawImage(image, 0, 0);
      };
const div = document.createElement('div');
const btnconnect = document.createElement('button');
const btndisconnect = document.createElement('button');
const capture = document.createElement('button');
                  
    async function takePhoto2(quality=1) {
      btnconnect.remove();
      capture.remove();
      device.ready_to_send_data = true;
    }


    async function takePhoto(quality=1) {

      btnconnect.textContent = 'connect';
      div.appendChild(btnconnect);
      btnconnect.onclick = this.connect;
      
      btndisconnect.textContent = 'disconnect';
      div.appendChild(btndisconnect);
      btndisconnect.onclick = this.disconnect;
      
      capture.textContent = 'Capture';
      capture.onclick = takePhoto2;
      div.appendChild(capture);
     
      div.appendChild(canvas);

      document.body.appendChild(div);
         await new Promise((resolve) => capture.onclick = resolve);
 
      btnconnect.remove();
      capture.remove();
      device.ready_to_send_data = true;
    }

    async function takePhoto1(quality=1) {  
      //var data_slice_send=this.device.data_slice;
      //var data_slice_send=[this.device.data_slice[0],this.device.data_slice[1]];
      //var data_slice_send=[this.device.data_slice[0]];
      var data_slice_send=[[this.device.data_slice[0][0]]];
	  	//console.log("data_slice_send[0].length:", data_slice_send[0].length);
  		//console.log("device.bufferednewLines:", device.bufferednewLines);
      device.bufferednewLines=0;
      return data_slice_send;      
    }

    async function displayPhoto1(photodata,photoWidth=300,photoHeight=300) {
      //if(canvas.width != photoWidth) canvas.width = photoWidth;
      //if(canvas.height != photoHeight) canvas.height = photoHeight;
      await displayPhoto2(photodata,photoWidth,photoHeight);
    }
    async function displayPhoto2(photodata,photoWidth,photoHeight) {
      if(canvas.width != photoWidth) canvas.width = photoWidth;
      if(canvas.height != photoHeight) canvas.height = photoHeight;
     image.src = photodata;
    }



  takePhoto();
  data_count=0;
  //device.ready_to_send_data = true;
  //while(1)
  //{
  //  data_count++;
  //  if(device.bufferednewLines>0)
  //  {
  //    channel.send('the data '+data_count+' '+device.bufferednewLines);
  //    device.bufferednewLines=0;
  //  }
  //} 

''')
  


<IPython.core.display.Javascript object>

In [22]:
stop

NameError: ignored

In [ ]:
stop

out.close()
from IPython import display as ipythondisplay
import io
import os
import base64
from IPython.display import HTML

def show_video(vid):
  ext = os.path.splitext(vid)[-1][1:]
  video = io.open(vid, 'r+b').read()
  ipythondisplay.display(HTML(data='''<video alt="test" autoplay 
              loop controls style="height: 400px;">
              <source src="data:video/{1}';base64,{0}" type="video/{1}" />
              </video>'''.format(base64.b64encode(video).decode('ascii'), ext)))

show_video('/content/out/output.mp4')

In [ ]:
from google.colab import files
files.download('/content/out/output.mp4')

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
!cp -r -v "/content/out" "/content/gdrive/MyDrive/EEG-GAN-audio-video"